### Examples Of Pyspark ML

In [1]:
import pyspark
from pyspark.sql import SparkSession
conf_spark = pyspark.SparkConf().set("spark.driver.host", "127.0.0.1")
sc = pyspark.SparkContext(conf=conf_spark)
spark=SparkSession.builder.appName('Missing').getOrCreate()


22/04/04 15:27:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 15:27:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/04 15:27:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/04/04 15:27:48 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/04/04 15:27:48 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/04/04 15:27:48 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [2]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()



+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [7]:
# [Age,Experience]----> new feature--->independent feature

In [9]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [10]:
output=featureassembler.transform(training)

In [11]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [12]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [13]:
finalized_data=output.select("Independent Features","Salary")

In [14]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [16]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

22/04/04 18:05:07 WARN Instrumentation: [22d77c59] regParam is zero, which might cause numerical instability and overfitting.


In [17]:
### Coefficients
regressor.coefficients

DenseVector([-64.8464, 1584.7554])

In [18]:
### Intercepts
regressor.intercept

15414.10693970376

In [19]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [20]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18612.059158134223|
+--------------------+------+------------------+



In [21]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1387.9408418657767, 1926379.780519081)